In [1]:
!pip install transformers faiss-cpu pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 57.7 MB/s eta 0:00:00


In [2]:
import faiss
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch


In [3]:
# Load the tokenizer and model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to generate embeddings
def get_embeddings(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the mean of the token embeddings as the sentence embedding
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.numpy()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [4]:
# Sample documents
documents = [
    "Artificial intelligence is transforming industries.",
    "Machine learning is a subset of AI focused on pattern recognition.",
    "FAISS is a library for efficient similarity search.",
    "Hugging Face provides pre-trained language models.",
    "Transformers are deep learning models revolutionizing NLP."
]

# Convert to embeddings
doc_embeddings = get_embeddings(documents)

# Create a mapping of document IDs to text
doc_map = {i: doc for i, doc in enumerate(documents)}

In [5]:
# Initialize FAISS index
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

print(f"Number of documents in the index: {index.ntotal}")

Number of documents in the index: 5


In [6]:
# Query the system
query = "What are transformers in AI?"
query_embedding = get_embeddings([query])

# Perform the search
k = 3  # Number of results to return
distances, indices = index.search(query_embedding, k)

# Display results
print("Query:", query)
print("\nTop results:")
for i, idx in enumerate(indices[0]):
    print(f"Result {i+1}: {doc_map[idx]} (Distance: {distances[0][i]:.2f})")


Query: What are transformers in AI?

Top results:
Result 1: Transformers are deep learning models revolutionizing NLP. (Distance: 34.39)
Result 2: Artificial intelligence is transforming industries. (Distance: 44.14)
Result 3: Machine learning is a subset of AI focused on pattern recognition. (Distance: 44.56)
